In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from gurobipy import *
import math

# 1

In [3]:
df = pd.read_csv('data_w_distance.csv', encoding='ISO-8859-1')

In [4]:
df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,Order City Coordinates,Customer City Coordinates,Distance_km
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class,"(-6.2349858, 106.9945444)","(18.2345399, -66.0351316)",18481.578516
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class,"(28.0159286, 73.3171367)","(18.2345399, -66.0351316)",13264.257819
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class,"(28.0159286, 73.3171367)","(37.3361663, -121.890591)",12577.000273
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class,"(-19.2569391, 146.8239537)","(34.0536909, -118.242766)",11630.156412
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class,"(-19.2569391, 146.8239537)","(18.2345399, -66.0351316)",16558.376450


In [5]:
df.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De

In [6]:
df_selected = df[['Order Item Id','Days for shipment (scheduled)','Order Item Product Price','Distance_km']].reset_index(drop=True)

In [7]:
df_selected.head()

,Order Item Id,Days for shipment (scheduled),Order Item Product Price,Distance_km
0,180517,4,327.75,18481.578516
1,179254,4,327.75,13264.257819
2,179253,4,327.75,12577.000273
3,179252,4,327.75,11630.156412
4,179251,4,327.75,16558.376450


In [8]:
df_selected = df_selected.sample(n=40000, random_state=42).reset_index(drop=True)

In [9]:
df_selected

,Order Item Id,Days for shipment (scheduled),Order Item Product Price,Distance_km
0,121434,2,50.000000,13188.692772
1,42440,1,399.980011,599.290561
2,18649,4,59.990002,6355.639404
3,94516,2,59.990002,1316.499593
4,20900,4,15.990000,14633.302886
...,...,...,...,...
39995,171085,4,49.980000,6985.470867
39996,154687,4,399.980011,9320.971905
39997,23398,4,399.980011,6617.883090
39998,90386,1,39.990002,7781.471311


# Gurobi optimization

In [10]:
shipping_speed = {
    'First_Class': 7000, #distance travel per day 
    'Same_Day': 5000,
    'Second_Class': 3000,
    'Standard_Class': 1000,
}

cost_factors = {
    'First_Class': 0.045,  
    'Same_Day': 0.025,
    'Second_Class': 0.015,
    'Standard_Class': 0.009,
}

penalty_per_day_late = 0.02

# Total number of orders
total_orders = len(df_selected)


In [11]:
#Initialize the model
m = Model("ShippingMode")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2411525
Academic license 2411525 - for non-commercial use only - registered to sh___@andrew.cmu.edu


In [12]:
#Setting up the parameters
orders = [i for i in range (total_orders)]
shipping_mode = [i for i in shipping_speed.keys()]

In [13]:
#Decision Variables. X[i,j] binary variable. 
x = m.addVars(orders, shipping_mode, vtype=GRB.BINARY)

In [14]:
#immediate shipping cost matrix. C[i,j] means the cost of shipping the ith order using jth shipping mode . 
#This should be calculated as the product price matching the ith order* cost_factors values for the chosen shipping mode.
cost = [
    [df_selected.loc[i, 'Order Item Product Price'] * cost_factors[shipping_mode[j]] for j in range(len(shipping_mode))]
    for i in range(len(orders))
]

immediate_cost = pd.DataFrame(cost, columns=shipping_mode)
immediate_cost

,First_Class,Same_Day,Second_Class,Standard_Class
0,2.25000,1.25000,0.75000,0.45000
1,17.99910,9.99950,5.99970,3.59982
2,2.69955,1.49975,0.89985,0.53991
3,2.69955,1.49975,0.89985,0.53991
4,0.71955,0.39975,0.23985,0.14391
...,...,...,...,...
39995,2.24910,1.24950,0.74970,0.44982
39996,17.99910,9.99950,5.99970,3.59982
39997,17.99910,9.99950,5.99970,3.59982
39998,1.79955,0.99975,0.59985,0.35991


In [15]:
#create a real delivery dataframe for the ith order having chosen jth shipping mode.
#The real delivery time is calculated as distance for delivering ith order/ shipping speed for jth shipping mode.
days = [
    [math.ceil(df_selected.loc[i, 'Distance_km'] / shipping_speed[shipping_mode[j]]) for j in range(len(shipping_mode))]
    for i in range(len(orders))
]
delivery_day = pd.DataFrame(days, columns=shipping_mode)
delivery_day

,First_Class,Same_Day,Second_Class,Standard_Class
0,2,3,5,14
1,1,1,1,1
2,1,2,3,7
3,1,1,1,2
4,3,3,5,15
...,...,...,...,...
39995,1,2,3,7
39996,2,2,4,10
39997,1,2,3,7
39998,2,2,3,8


In [16]:
#create a late day dataframe
#late days are calculated as actual delivery days- scheduled delivery days
late_days = [
    [max(0,delivery_day.loc[i,shipping_mode[j]]- int(df_selected.loc[i,"Days for shipment (scheduled)"]))for j in range(len(shipping_mode))]
    for i in orders
]
late_day= pd.DataFrame(late_days, columns=shipping_mode)
late_day

,First_Class,Same_Day,Second_Class,Standard_Class
0,0,1,3,12
1,0,0,0,0
2,0,0,0,3
3,0,0,0,0
4,0,0,1,11
...,...,...,...,...
39995,0,0,0,3
39996,0,0,0,6
39997,0,0,0,3
39998,1,1,2,7


In [17]:
#create a late delivery cost for ith order having chosen jth shipping mode.
#the value will be late_days* penalty_per_day_late*product price.
late_delivery_cost = [[late_day.loc[i,shipping_mode[j]]*penalty_per_day_late*df_selected.loc[i,"Order Item Product Price"]
                    for j in range(len(shipping_mode))]
                    for i in range(len(orders))
                    ]
late_cost = pd.DataFrame(late_delivery_cost, columns=shipping_mode)
late_cost


,First_Class,Same_Day,Second_Class,Standard_Class
0,0.0000,1.0000,3.0000,12.000000
1,0.0000,0.0000,0.0000,0.000000
2,0.0000,0.0000,0.0000,3.599400
3,0.0000,0.0000,0.0000,0.000000
4,0.0000,0.0000,0.3198,3.517800
...,...,...,...,...
39995,0.0000,0.0000,0.0000,2.998800
39996,0.0000,0.0000,0.0000,47.997601
39997,0.0000,0.0000,0.0000,23.998801
39998,0.7998,0.7998,1.5996,5.598600


In [18]:
#The objective is to minimize the total cost associated with choosing the specific shipping mode for each order.
#the total cost is the late delivery cost+ immediate_cost
m.setObjective(
    quicksum(immediate_cost.loc[i, shipping_mode[j]]*x[i,shipping_mode[j]] + late_cost.loc[i, shipping_mode[j]]*x[i,shipping_mode[j]] for j in range(len(shipping_mode)) for i in orders),
    GRB.MINIMIZE
)


In [19]:
# set constraints 

#first constraint is that for each order (for all i), there should only be one shipping mode chosen.
m.addConstrs(quicksum(x[i, shipping_mode[j]] for j in range(len(shipping_mode))) == 1 for i in orders)

#second constraint is that for each shipping mode, it has a maximum delivery capacity of total_orders / 4
m.addConstrs(quicksum(x[i, shipping_mode[j]] for i in orders) <= total_orders/4 for j in range(len(shipping_mode)))

# #third constraint is that the maximum late delivery day should be less than 5 days to ensure that the scheduled days are not overly underestimated.
m.addConstrs(late_day.loc[i,shipping_mode[j]]*x[i,shipping_mode[j]] <= 5 for i in orders for j in range(len(shipping_mode)))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


In [20]:
m.update()

In [21]:
# Solve
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900HK, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Academic license 2411525 - for non-commercial use only - registered to sh___@andrew.cmu.edu
Optimize a model with 200004 rows, 160000 columns and 382056 nonzeros
Model fingerprint: 0x59753144
Variable types: 0 continuous, 160000 integer (160000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [9e-02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 160254 rows and 17539 columns
Presolve time: 0.37s
Presolved: 39750 rows, 142461 columns, 284922 nonzeros
Variable types: 0 continuous, 142461 integer (142460 binary)
Found heuristic solution: objective 253713.76037
Found heuristic solution: objective 253004.27856
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only

In [22]:
print("Retrieve Optimal Solution.")

# Print objective value
print("Objective value =", m.objVal)

print("***")


Retrieve Optimal Solution.
Objective value = 146290.01909771963
***


In [23]:
#print out the optimal choice dataframe
selected_modes = [(i, j) for i in orders for j in shipping_mode if x[i, j].x == 1]
optimal_choice = pd.DataFrame(selected_modes, columns=['Order','Selected_Shipping_Mode'])
optimal_choice.set_index('Order', inplace=True)
optimal_choice = pd.merge(df_selected["Order Item Id"], optimal_choice, left_index=True, right_index=True)
optimal_choice

,Order Item Id,Selected_Shipping_Mode
0,121434,First_Class
1,42440,Standard_Class
2,18649,Same_Day
3,94516,Standard_Class
4,20900,First_Class
...,...,...
39995,171085,First_Class
39996,154687,Second_Class
39997,23398,Second_Class
39998,90386,First_Class


In [28]:
from optiguide.optiguide import OptiGuideAgent
from flaml.autogen.agentchat import UserProxyAgent

In [29]:
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

In [30]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)

In [27]:
your_model_code_as_string = """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from gurobipy import *
import math
df = pd.read_csv('data_w_distance.csv', encoding='ISO-8859-1')
df.head()
df.columns
df_selected = df[['Order Item Id','Days for shipment (scheduled)','Order Item Product Price','Distance_km']].reset_index(drop=True)
df_selected.head()
df_selected = df_selected.sample(n=40000, random_state=42).reset_index(drop=True)
df_selected
shipping_speed = {
    'First_Class': 7000, #distance travel per day 
    'Same_Day': 5000,
    'Second_Class': 3000,
    'Standard_Class': 1000,
}

cost_factors = {
    'First_Class': 0.045,  
    'Same_Day': 0.025,
    'Second_Class': 0.015,
    'Standard_Class': 0.009,
}

penalty_per_day_late = 0.02

# Total number of orders
total_orders = len(df_selected)
m = Model("ShippingMode")
orders = [i for i in range (total_orders)]
shipping_mode = [i for i in shipping_speed.keys()]
x = m.addVars(orders, shipping_mode, vtype=GRB.BINARY)
cost = [
    [df_selected.loc[i, 'Order Item Product Price'] * cost_factors[shipping_mode[j]] for j in range(len(shipping_mode))]
    for i in range(len(orders))
]

immediate_cost = pd.DataFrame(cost, columns=shipping_mode)
immediate_cost
days = [
    [math.ceil(df_selected.loc[i, 'Distance_km'] / shipping_speed[shipping_mode[j]]) for j in range(len(shipping_mode))]
    for i in range(len(orders))
]
delivery_day = pd.DataFrame(days, columns=shipping_mode)
delivery_day
late_days = [
    [max(0,delivery_day.loc[i,shipping_mode[j]]- int(df_selected.loc[i,"Days for shipment (scheduled)"]))for j in range(len(shipping_mode))]
    for i in orders
]
late_day= pd.DataFrame(late_days, columns=shipping_mode)
late_day
late_delivery_cost = [[late_day.loc[i,shipping_mode[j]]*penalty_per_day_late*df_selected.loc[i,"Order Item Product Price"]
                    for j in range(len(shipping_mode))]
                    for i in range(len(orders))
                    ]
late_cost = pd.DataFrame(late_delivery_cost, columns=shipping_mode)
late_cost
m.setObjective(
    quicksum(immediate_cost.loc[i, shipping_mode[j]]*x[i,shipping_mode[j]] + late_cost.loc[i, shipping_mode[j]]*x[i,shipping_mode[j]] for j in range(len(shipping_mode)) for i in orders),
    GRB.MINIMIZE
)
#first constraint is that for each order (for all i), there should only be one shipping mode chosen.
m.addConstrs(quicksum(x[i, shipping_mode[j]] for j in range(len(shipping_mode))) == 1 for i in orders)

#second constraint is that for each shipping mode, it has a maximum delivery capacity of total_orders / 4
m.addConstrs(quicksum(x[i, shipping_mode[j]] for i in orders) <= total_orders/4 for j in range(len(shipping_mode)))

# #third constraint is that the maximum late delivery day should be less than 5 days to ensure that the scheduled days are not overly underestimated.
m.addConstrs(late_day.loc[i,shipping_mode[j]]*x[i,shipping_mode[j]] <= 5 for i in orders for j in range(len(shipping_mode)))
m.update()
m.optimize()
print("Retrieve Optimal Solution.")

# Print objective value
print("Objective value =", m.objVal)

print("***")

#print out the optimal choice dataframe
selected_modes = [(i, j) for i in orders for j in shipping_mode if x[i, j].x == 1]
optimal_choice = pd.DataFrame(selected_modes, columns=['Order','Selected_Shipping_Mode'])
optimal_choice.set_index('Order', inplace=True)
optimal_choice = pd.merge(df_selected["Order Item Id"], optimal_choice, left_index=True, right_index=True)
optimal_choice
"""

agent = OptiGuideAgent(
    name="GurobiOptimizationModel",
    source_code=your_model_code_as_string,
    debug_times=1,
    example_qa="",
    llm_config={"request_timeout": 600, "seed": 42}
)

AssertionError: DATA_CODE_STR not found.